In [1]:
import logging
import pickle

from pathlib import Path
from random import choice
from timeit import default_timer as timer

from geoloc_imc_2023.cbg import CBG, get_prefix_from_ip
from geoloc_imc_2023.query_api import retreive_single_measurement

from geoloc_imc_2023.default import (
    PROBES_PATH,
    RESULTS_PATH,
    HITLIST_PATH,
)

logging.basicConfig(level=logging.INFO)

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

NB_TARGET = 3
NB_VP = 3

In [2]:
# load hitlist
with open(HITLIST_PATH, "rb") as f:
    targets_per_prefix = pickle.load(f)

# load anchors
with open(PROBES_PATH, "rb") as f:
    probes = pickle.load(f)

## select target and vps dataset

In [3]:
# select targets and vps from anchors
targets= list(probes.keys())[:NB_TARGET]
vps = {}
for i, probe in enumerate(probes):
    if i >= NB_VP: break
    vps[probe] = probes[probe]

logging.info(f"nb targets: {len(targets)} : {[target for target in targets]}")
logging.info(f"nb_vps : {len(vps)} {[vp for vp in vps]}")

INFO:root:nb targets: 3 : ['45.138.229.91', '77.174.76.85', '82.217.219.124']
INFO:root:nb_vps : 3 ['45.138.229.91', '77.174.76.85', '82.217.219.124']


## Tier 1: probing each target prefixes

In [4]:
# use all probes as targets and all probes as vps
cbg = CBG(ripe_credentials)

# get target prefixes
target_prefixes = []
for target_addr in targets:
    target_prefix = get_prefix_from_ip(target_addr)
    target_prefixes.append(target_prefix)

# initialization: start measurements from all vps to targets
init_measurement_ids = cbg.prefix_probing(target_prefixes, vps, targets_per_prefix, dry_run=True)

# retreive measurement results from RIPE Atlas
logging.info(init_measurement_ids)

TypeError: CBG.prefix_probing() missing 1 required positional argument: 'target_hitlist'

In [2]:
# save results
out_file = f"../results/init_measurement_ids.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(init_measurement_ids,f)

../results/init_measurement_ids.pickle


## Retrieve prefix probing results 

In [1]:
measurement_ids_file = Path(".") / "../results/init_measurement_ids.pickle"
with open(measurement_ids_file, "rb") as f:
    init_measurement_ids = pickle.load(f)

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

measurement_results = {}
for prefix in init_measurement_ids:
    for measurement_id in init_measurement_ids[prefix]:
        measurement_results[prefix] = retreive_single_measurement(measurement_id, anchors, ripe_credentials["key"])

In [2]:
# save results
out_file = f"../results/init_measurement.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)

../results/init_measurement.pickle


## Tier 2: probe each target

In [1]:
ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

# load hitlist
hitlist_file = Path(".") / "../datasets/parsed_hitlist.pickle"
with open(hitlist_file, "rb") as f:
    hitlist_addr = pickle.load(f)

targets = [anchor_addr for anchor_addr in anchors][:NB_TARGET]
vps = [anchor_addr for anchor_addr in anchors][:NB_VP]

logging.info(f"nb targets: {len(targets)}")
logging.info(f"nb_vps : {len(vps)}")

In [ ]:
cbg = CBG(targets, vps, ripe_credentials, hitlist_addr, anchors)

start = timer()
measurement_ids = await cbg.target_probing(targets, vps)
end =timer()

time_elapsed = end - start
print(time_elapsed)

In [ ]:
# save results
out_file = f"../results/target_measurement_ids.pickle"
with open(out_file, "wb") as f:
    pickle.dump(measurement_ids,f)

## Retreive target measurements

In [ ]:
# load hitlist
in_file = Path(".") / "../results/target_measurement_ids.pickle"
with open(in_file, "rb") as f:
    measurement_ids = pickle.load(f)

# load anchors
anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

print(len(measurement_ids))

measurement_results = []
for measurement_id in measurement_ids:
    measurement_result = retreive_single_measurement(measurement_id, anchors, ripe_credentials["key"])
    print(f"got results for target: {measurement_result['target_addr']} from {len(measurement_result['results'])} servers")
    measurement_results.append(measurement_result)

In [15]:
# save results
out_file = f"../results/target_measurement.pickle"
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results,f)

../results/target_measurement.pickle
